1 - Instalar dependencias

In [1]:
!pip install -U langchain langchain-openai openai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52


2 - Crear la ase de datos SQLite en memoria

In [2]:
import sqlite3

sqlite_conn = sqlite3.connect(":memory:", check_same_thread=False)
sqlite_conn.row_factory = sqlite3.Row

# Crear tablas
sqlite_conn.executescript("""
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT
);

CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    name TEXT,
    price REAL
);

CREATE TABLE orders (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    order_date DATETIME,
    FOREIGN KEY(user_id) REFERENCES users(id),
    FOREIGN KEY(product_id) REFERENCES products(id)
);
""")

# Insertar datos
sqlite_conn.executemany("INSERT INTO users (id, name, email) VALUES (?, ?, ?)", [
    (1, "Juan Pérez", "juan@example.com"),
    (2, "Ana Gómez", "ana@example.com"),
    (3, "Luis Torres", "luis@example.com")
])

sqlite_conn.executemany("INSERT INTO products (id, name, price) VALUES (?, ?, ?)", [
    (1, "Laptop", 999.99),
    (2, "Teclado", 49.99),
    (3, "Mouse", 25.99)
])

sqlite_conn.executemany("INSERT INTO orders (id, user_id, product_id, quantity, order_date) VALUES (?, ?, ?, ?, datetime('now', ?))", [
    (1, 1, 1, 1, "-3 days"),
    (2, 1, 2, 2, "-2 days"),
    (3, 2, 3, 3, "-1 days"),
    (4, 3, 1, 1, "-7 days")
])

sqlite_conn.commit()


3 - Inicializar el modelo LLM y el parser

In [3]:
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
import os
import getpass

# Tu clave OpenAI
#os.environ["OPENAI_API_KEY"] = "sk-..."  # Reemplaza con tu clave
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

# Esquema de tablas
table_schema = """
Table users:
  - id (INTEGER)
  - name (TEXT)
  - email (TEXT)

Table products:
  - id (INTEGER)
  - name (TEXT)
  - price (REAL)

Table orders:
  - id (INTEGER)
  - user_id (INTEGER) → users.id
  - product_id (INTEGER) → products.id
  - quantity (INTEGER)
  - order_date (DATETIME)
"""

# Salida esperada
class QueryOutput(BaseModel):
    query: str = Field(..., description="Syntactically valid SQL query.")

# LLM + parser
llm = ChatOpenAI(model="gpt-4o", temperature=0)
parser = PydanticOutputParser(pydantic_object=QueryOutput)

# Prompt con instrucciones explícitas de formato JSON
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un generador experto de SQL. Devuelve solo JSON con el siguiente formato:\n\n{format_instructions}\n\n"
               "No incluyas comentarios ni formato Markdown. Solo el JSON."),
    ("user", "Dialecto: {dialect}\n\nEsquema:\n{table_info}\n\nPregunta: {input}")
])


Enter API key for OpenAI: ··········


4 - Funciones para generar y ejecutar la consulta

In [4]:
def write_query(natural_question: str, schema: str, dialect: str = "sqlite") -> str:
    format_instructions = parser.get_format_instructions()
    prompt = prompt_template.invoke({
        "dialect": dialect,
        "table_info": schema,
        "input": natural_question,
        "format_instructions": format_instructions
    })
    response = llm.invoke(prompt)
    result = parser.parse(response.content)
    return result.query

def execute_sqlite_query(query: str, conn: sqlite3.Connection):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        return [dict(row) for row in rows]
    except Exception as e:
        return [{"error": str(e)}]


Preguntas:

"¿Qué usuarios hicieron pedidos en los últimos 5 días?"

"¿Cuántos productos pidió cada usuario?"

"Muestra los pedidos con nombre de producto y nombre de usuario"

"Total gastado por cada usuario"

5 - Crear interfaz Gradio

In [5]:
import gradio as gr
import pandas as pd

def run_query_interface(natural_question):
    try:
        sql = write_query(natural_question, table_schema)
        rows = execute_sqlite_query(sql, sqlite_conn)
        df = pd.DataFrame(rows)
        return sql, df
    except Exception as e:
        return f"❌ Error: {str(e)}", pd.DataFrame()

demo = gr.Interface(
    fn=run_query_interface,
    inputs=gr.Textbox(label="Pregunta en lenguaje natural", placeholder="¿Qué usuarios se registraron en los últimos 7 días?"),
    outputs=[
        gr.Textbox(label="Consulta SQL generada"),
        gr.Dataframe(label="Resultados")
    ],
    title="💬 GPT-4o a SQL con SQLite",
    description="Haz preguntas en lenguaje natural sobre la tabla `users` y GPT generará la consulta SQL y mostrará los resultados."
)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2ffeea5e7f97c9a574.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
